In [10]:
import pandas as pd
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing
from urllib.parse import urlparse
import warnings as wrn
wrn.filterwarnings(action="ignore")

In [3]:
housing = fetch_california_housing()

data = pd.DataFrame(housing.data, columns=housing.feature_names)
data['Price'] = housing.target

data.head(3)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521


In [8]:
# hyperparameter tuning using Grid Searchcv
def hyperparameter_tuning(X_train, y_train, param_grid):
    """
        Hyperparameter tuning using GridSearchCV on a RandomForestRegressor
        
        Parameters:
            X_train (2d array): Feature metrix for training.
            y_train (array): Target variable.
            param_grid (dict): Hyperparameter values to search.
        
        Returns:
            GridSearchCV: The fitted GridSearchCV object containing the best parameters.
    """
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(
        estimator=rf,
        param_grid=param_grid,
        cv=3,
        n_jobs=-1,
        verbose=1,
        scoring="neg_mean_squared_error"
    )
    grid_search.fit(X_train, y_train)
    return grid_search

In [9]:
X = data.drop(columns=["Price"])
y = data["Price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(16512, 8) (16512,)
(4128, 8) (4128,)


In [11]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

In [13]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

signature = infer_signature(X_train, y_train)

with mlflow.start_run():
    grid_search = hyperparameter_tuning(X_train, y_train, param_grid)
    
    best_model = grid_search.best_estimator_
    print(f"best model: {best_model}")
    
    y_pred = best_model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    
    mlflow.log_param("best_n_estimators", grid_search.best_params_['n_estimators'])
    mlflow.log_param("best_max_depth", grid_search.best_params_['max_depth'])
    mlflow.log_param("best_min_samples_split", grid_search.best_params_['min_samples_split'])
    mlflow.log_param("best_min_samples_leaf", grid_search.best_params_['min_samples_leaf'])
    mlflow.log_metric("MSE", mse)
    
    # mlflow.set_tracking_uri("http://127.0.0.1:5000")
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme # it will return 'http' for localhost, if no tracking_uri is set then file
    
    if tracking_url_type_store != 'file':
        mlflow.sklearn.log_model(best_model, "model", registered_model_name="Best RandomForest Model")
    else:
        mlflow.sklearn.log_model(best_model, "model", signature=signature)
        
    print(f"Best Hyperparameters: {grid_search.best_params_}")
    print(f"Mean Squared Error: {mse}")

Fitting 3 folds for each of 24 candidates, totalling 72 fits
best model: RandomForestRegressor(min_samples_leaf=2, n_estimators=200)


2025/03/07 20:55:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Best RandomForest Model'.
2025/03/07 20:55:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best RandomForest Model, version 1


Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Mean Squared Error: 0.24281032254951324
🏃 View run smiling-swan-951 at: http://127.0.0.1:5000/#/experiments/0/runs/a889f1ded0db4f088617d48e935cf661
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


Created version '1' of model 'Best RandomForest Model'.


# Make Predictions

In [18]:
logged_model = 'runs:/a889f1ded0db4f088617d48e935cf661/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

loaded_model.predict(pd.DataFrame(X_test))

array([1.58990792, 1.26250508, 2.42719615, ..., 1.37708991, 1.30560279,
       1.58006784], shape=(4128,))